# **Setup**

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
# print(tensorflow.version)

In [3]:


# !{sys.executable} -m pip install tensorflow==2.6.0rc0 -q

In [4]:
# %tensorflow_version 2.x
import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [5]:
print(tf. __version__)

2.6.0


In [6]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [7]:
# import sys
# !{sys.executable} -m pip install mediapipe
!pip install mediapipe


In [8]:
# !{sys.executable} -m pip install DynamiKontrol

In [9]:
!pip install opencv-python==4.5.5.64


In [10]:
import numpy as np
import cv2
from cv2 import *
import tensorflow
import mediapipe as mp
import time
import glob
# from google.colab.patches import cv2_imshow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

In [11]:
# # the project folder is saved in my google drive
# %cp -av "/content/gdrive/MyDrive/loopQ/project" "/content"

In [12]:
# %cd /content/project

# **Parameters & Model**

In [13]:
emotions = {
    0: ['Angry', (0,0,255), (255,255,255)],
    1: ['Disgust', (0,102,0), (255,255,255)],
    2: ['Fear', (255,255,153), (0,51,51)],
    3: ['Happy', (153,0,153), (255,255,255)],
    4: ['Sad', (255,0,0), (255,255,255)],
    5: ['Surprise', (0,255,0), (255,255,255)],
    6: ['Neutral', (160,160,160), (255,255,255)]
}
num_classes = len(emotions)
input_shape = (48, 48, 1)
weights_1 = 'saved_models/vggnet.h5'
weights_2 = 'saved_models/vggnet_up.h5'

In [14]:
loaded_model = tensorflow.keras.models.load_model("Final_model_ck+_mobilenetv2.h5")

# **Inference**

In [15]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

In [16]:
def detection_preprocessing(image, h_max=360):
    h, w, _ = image.shape
    if h > h_max:
        ratio = h_max / h
        w_ = int(w * ratio)
        image = cv2.resize(image, (w_,h_max))
    return image

def resize_face(face):
    # x = tf.expand_dims(tf.convert_to_tensor(face), axis=2)
    x = tf.convert_to_tensor(face)
    rgb_batch = np.repeat(x[..., np.newaxis], 3, -1)
    # print(rgb_batch.shape)
    x = tf.image.resize(rgb_batch, (48,48))
    # backtorgb = cv2.cvtColor(x,cv2.COLOR_GRAY2RGB)
    # print(backtorgb.shape)
    # print(x.shape)
    
    return x

def recognition_preprocessing(faces):
    x = tf.convert_to_tensor([resize_face(f) for f in faces])
    return x

In [17]:
from cv2 import VideoCapture
from cv2 import imshow
from cv2 import imwrite
from cv2 import waitKey
from cv2 import destroyWindow

In [18]:
def inference(image):
    H, W, _ = image.shape
    
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # print(rgb_image.shape)
    results = face_detection.process(rgb_image)

    if results.detections:
        faces = []
        pos = []
        for detection in results.detections:
            box = detection.location_data.relative_bounding_box
            # mp_drawing.draw_detection(image, detection)

            x = int(box.xmin * W)
            y = int(box.ymin * H)
            w = int(box.width * W)
            h = int(box.height * H)

            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(x + w, W)
            y2 = min(y + h, H)

            face = image[y1:y2,x1:x2]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            faces.append(face)
            pos.append((x1, y1, x2, y2))
    
        x = recognition_preprocessing(faces)

        y_1 = loaded_model.predict(x)
        # y_2 = model_2.predict(x)
        print(y_1)
        l = np.argmax(y_1, axis=1)

        for i in range(len(faces)):
            cv2.rectangle(image, (pos[i][0],pos[i][1]),
                            (pos[i][2],pos[i][3]), emotions[l[i]][1], 2, lineType=cv2.LINE_AA)
            
            cv2.rectangle(image, (pos[i][0],pos[i][1]-20),
                            (pos[i][2]+20,pos[i][1]), emotions[l[i]][1], -1, lineType=cv2.LINE_AA)
            
            cv2.putText(image, f'{emotions[l[i]][0]}', (pos[i][0],pos[i][1]-5),
                            0, 0.6, emotions[l[i]][2], 2, lineType=cv2.LINE_AA)
    
    return image

## **Image**

In [19]:
# def infer_single_image(path):
#     image = cv2.imread(path)
#     image = detection_preprocessing(image)
#     result = inference(image)
#     cv2.imwrite('run/inference/out.jpg', result)

# # def infer_multi_images(paths):
# #     for i, path in enumerate(paths):
# #         image = cv2.imread(path)
# #         image = detection_preprocessing(image)
# #         result = inference(image)
# #         cv2.imwrite('run/inference/out_'+str(i)+'.jpg', result)

In [20]:
# infer_single_image('test.jpg')
# out = cv2.imread('run/inference/out.jpg')
# imshow(out)

In [21]:
# paths = np.sort(np.array(glob.glob('images/*.jpg')))
# infer_multi_images(paths)
# out_paths = np.sort(np.array(glob.glob('run/inference/*.jpg')))
# for path in out_paths:
#     image = cv2.imread(path)
#     cv2_imshow(image)

In [22]:
# %cp -av /content/project/run/inference/ /content/gdrive/MyDrive/loopQ/project/run/

## **Video**

In [23]:
# video = 'test_video/emotions.mp4'
# cap = cv2.VideoCapture(video)
# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))
# fps = cap.get(cv2.CAP_PROP_FPS)
# target_h = 360
# target_w = int(target_h * frame_width / frame_height)
# out = cv2.VideoWriter('run/out.avi',cv2.VideoWriter_fourcc('M','J','P','G'),
#                       fps, (target_w,target_h))

In [24]:
# while True:
#     success, image = cap.read()
#     if success:
#         image = resize_image(image)
#         result = inference(image)
#         out.write(result)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     else:
#         break
    
# cap.release()
# out.release()
# cv2.destroyAllWindows()

# **Cam**

In [25]:
import cv2
# import dynamikontrol
# from dynamikontrol import Module

In [26]:
from cv2 import VideoCapture
from cv2 import imshow
from cv2 import imwrite
from cv2 import waitKey
from cv2 import destroyWindow

In [27]:
from time import sleep

In [28]:
import os
import cv2
import time


# init camera
execution_path = os.getcwd()
camera = cv2.VideoCapture(0)

while True:
    # Init and FPS process
    start_time = time.time()

    # Grab a single frame of video
    ret, frame = camera.read()

    val = inference(frame)



    # calculate FPS >> FPS = 1 / time to process loop
#     fpsInfo = "FPS: " + str(1.0 / (time.time() - start_time)) 
#     print(fpsInfo)


    # cv2.putText(frame, fpsInfo, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    

    # Display the resulting image
    cv2.imshow('Video', val)

    # sleep(100)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
camera.release()
cv2.destroyAllWindows()

[[3.7137692e-05 4.9877621e-04 2.4608737e-03 9.3277509e-04 2.9313082e-03
  7.6787161e-05 9.9306232e-01]]
[[3.6246030e-04 4.0245242e-02 2.9077253e-03 8.0213970e-01 4.2266831e-02
  4.9075123e-04 1.1158728e-01]]
[[3.6246030e-04 4.0245242e-02 2.9077253e-03 8.0213970e-01 4.2266831e-02
  4.9075123e-04 1.1158728e-01]]
[[1.4631264e-04 1.6090447e-02 1.5533721e-03 9.0629095e-01 3.7318125e-02
  2.1688890e-04 3.8383905e-02]]
[[1.4631264e-04 1.6090447e-02 1.5533721e-03 9.0629095e-01 3.7318125e-02
  2.1688890e-04 3.8383905e-02]]
[[5.3161709e-04 5.0907191e-02 3.4249949e-03 6.3225061e-01 3.0643178e-02
  8.9387619e-04 2.8134853e-01]]
[[2.2818366e-04 7.8841761e-02 1.7601267e-03 7.8910762e-01 5.2927036e-02
  2.5821361e-04 7.6877013e-02]]
[[2.2818366e-04 7.8841761e-02 1.7601267e-03 7.8910762e-01 5.2927036e-02
  2.5821361e-04 7.6877013e-02]]
[[0.00061575 0.11843111 0.00397769 0.50392103 0.04943727 0.00075821
  0.32285887]]
[[0.00061575 0.11843111 0.00397769 0.50392103 0.04943727 0.00075821
  0.32285887]]
[[

[[1.6959396e-04 3.0386102e-04 7.0100706e-03 1.2969553e-03 1.1747918e-03
  2.3709130e-04 9.8980767e-01]]
[[9.8542645e-05 6.0873595e-04 3.5208480e-03 2.0640264e-03 9.5077045e-03
  5.6038110e-04 9.8363978e-01]]
[[8.1779726e-05 2.2104716e-04 1.4592382e-03 3.9341292e-04 2.5447103e-04
  6.6309312e-04 9.9692696e-01]]
[[8.1779726e-05 2.2104716e-04 1.4592382e-03 3.9341292e-04 2.5447103e-04
  6.6309312e-04 9.9692696e-01]]
[[3.6303297e-04 6.0623384e-04 6.1483006e-03 7.3835487e-04 7.8770093e-04
  5.7933554e-03 9.8556292e-01]]
[[3.6303297e-04 6.0623384e-04 6.1483006e-03 7.3835487e-04 7.8770093e-04
  5.7933554e-03 9.8556292e-01]]
[[1.16074625e-04 2.60584755e-04 7.42122065e-03 4.44215635e-04
  2.25661346e-03 5.34938648e-04 9.88966405e-01]]
[[1.16074625e-04 2.60584755e-04 7.42122065e-03 4.44215635e-04
  2.25661346e-03 5.34938648e-04 9.88966405e-01]]
[[3.3141954e-05 1.2651616e-04 2.8376922e-03 2.0344841e-04 1.9080874e-03
  1.3767793e-04 9.9475336e-01]]
[[3.3141954e-05 1.2651616e-04 2.8376922e-03 2.0344

[[8.93784454e-05 1.13900205e-04 1.69657334e-03 3.45329434e-04
  1.81870069e-04 9.06382804e-04 9.96666610e-01]]
[[1.7522214e-05 7.4418684e-05 1.8241181e-03 1.4247246e-04 5.3829188e-04
  4.2778436e-05 9.9736041e-01]]
[[1.7522214e-05 7.4418684e-05 1.8241181e-03 1.4247246e-04 5.3829188e-04
  4.2778436e-05 9.9736041e-01]]
[[1.7522214e-05 7.4418684e-05 1.8241181e-03 1.4247246e-04 5.3829188e-04
  4.2778436e-05 9.9736041e-01]]
[[8.9112491e-06 1.5203440e-05 5.9274741e-04 6.3309075e-05 5.8871094e-05
  2.7846445e-05 9.9923313e-01]]
[[5.5250377e-05 3.6882251e-04 3.0178358e-03 3.1970779e-04 7.5115537e-04
  2.0903138e-04 9.9527824e-01]]
[[3.2242507e-04 7.3989676e-03 5.0664661e-03 5.8972865e-04 2.4731799e-03
  8.9160877e-04 9.8325759e-01]]
[[3.9230386e-04 1.8521545e-03 3.4460700e-03 6.6325581e-04 8.1257947e-04
  8.3847303e-04 9.9199510e-01]]
[[0.0138772  0.05468557 0.03581782 0.0153848  0.01204956 0.01468017
  0.8535049 ]]
[[0.02220431 0.08817603 0.04779829 0.03322257 0.02051031 0.03542405
  0.752664

[[6.2527542e-06 1.0160649e-05 1.2255721e-03 2.8640343e-05 1.2485310e-04
  3.1114956e-05 9.9857342e-01]]
[[6.3229181e-06 5.8235673e-06 7.4875995e-04 5.2416461e-05 4.6633235e-05
  2.8471493e-05 9.9911147e-01]]
[[6.3229181e-06 5.8235673e-06 7.4875995e-04 5.2416461e-05 4.6633235e-05
  2.8471493e-05 9.9911147e-01]]
[[4.1312223e-06 4.4117019e-06 1.0526683e-03 3.5540335e-05 9.2539412e-05
  2.4886131e-05 9.9878591e-01]]
[[1.7566972e-06 3.0148008e-06 5.1860494e-04 1.9943242e-05 1.2035667e-04
  7.3499441e-06 9.9932897e-01]]
[[1.7566972e-06 3.0148008e-06 5.1860494e-04 1.9943242e-05 1.2035667e-04
  7.3499441e-06 9.9932897e-01]]
[[1.5051779e-06 3.5138767e-06 4.8138370e-04 2.2453998e-05 2.0499997e-04
  6.2947643e-06 9.9927980e-01]]
[[1.5051779e-06 3.5138767e-06 4.8138370e-04 2.2453998e-05 2.0499997e-04
  6.2947643e-06 9.9927980e-01]]
[[4.5018031e-05 6.7164714e-04 5.8251796e-03 4.0116995e-03 1.6046049e-02
  9.9898200e-05 9.7330058e-01]]
[[4.5018031e-05 6.7164714e-04 5.8251796e-03 4.0116995e-03 1.6046

[[3.4992973e-04 1.3403206e-03 5.0028460e-03 1.8081469e-02 1.4552185e-03
  6.0592481e-04 9.7316426e-01]]
[[0.00150086 0.00090223 0.06504905 0.04013831 0.00493164 0.0012829
  0.88619506]]
[[0.00150086 0.00090223 0.06504905 0.04013831 0.00493164 0.0012829
  0.88619506]]
[[0.01577315 0.24556538 0.03710791 0.00662747 0.02884803 0.00384193
  0.6622361 ]]
[[0.01577315 0.24556538 0.03710791 0.00662747 0.02884803 0.00384193
  0.6622361 ]]
[[0.00130152 0.25978366 0.01449396 0.02184498 0.11368047 0.00076427
  0.5881311 ]]
[[0.00130152 0.25978366 0.01449396 0.02184498 0.11368047 0.00076427
  0.5881311 ]]
[[2.6294374e-05 2.5394559e-04 1.8435010e-03 1.3329822e-04 5.6810613e-04
  6.0979437e-05 9.9711394e-01]]
[[1.9439813e-05 2.7137928e-04 1.0662618e-03 8.4487729e-05 4.1403016e-04
  4.3180637e-05 9.9810117e-01]]
[[1.9439813e-05 2.7137928e-04 1.0662618e-03 8.4487729e-05 4.1403016e-04
  4.3180637e-05 9.9810117e-01]]
[[9.3911831e-06 1.6805965e-05 4.2033827e-04 2.0435727e-05 2.5305777e-05
  3.8322287e-05 

...

[[0.01336822 0.03784363 0.16387886 0.01480148 0.04662943 0.00127974
  0.7221987 ]]
[[0.00614182 0.27343935 0.01992037 0.00660775 0.02755808 0.00089516
  0.66543746]]
[[3.9978605e-03 3.4507401e-02 8.6385764e-02 1.9316079e-02 4.4626914e-02
  5.3037313e-04 8.1063563e-01]]
[[3.9978605e-03 3.4507401e-02 8.6385764e-02 1.9316079e-02 4.4626914e-02
  5.3037313e-04 8.1063563e-01]]
[[0.00467816 0.6491713  0.00836624 0.01133533 0.02455856 0.00095265
  0.30093783]]
[[0.00467816 0.6491713  0.00836624 0.01133533 0.02455856 0.00095265
  0.30093783]]
[[1.6364615e-03 9.5154345e-01 8.4767159e-04 5.7142059e-04 4.6755741e-03
  4.1814559e-04 4.0307168e-02]]
[[0.00425446 0.7429294  0.00540822 0.00285998 0.01677714 0.00078451
  0.22698627]]
[[0.00425446 0.7429294  0.00540822 0.00285998 0.01677714 0.00078451
  0.22698627]]
[[3.5090607e-03 7.1364504e-01 4.9302424e-03 3.6624356e-03 1.7254282e-02
  6.3916080e-04 2.5635979e-01]]
[[3.5090607e-03 7.1364504e-01 4.9302424e-03 3.6624356e-03 1.7254282e-02
  6.3916080e-0

[[5.8703317e-05 9.9511009e-01 1.5404050e-05 2.0207296e-04 1.2743312e-03
  3.6690955e-05 3.3026759e-03]]
[[3.1876756e-04 7.9046965e-01 7.5492513e-04 3.2202076e-02 2.6822176e-02
  2.9530958e-04 1.4913711e-01]]
[[1.8466030e-04 1.1757346e-02 1.5463027e-02 1.0789919e-01 1.0340543e-01
  1.6633085e-04 7.6112401e-01]]
[[3.5067971e-04 7.4422508e-02 6.6936724e-03 3.9359331e-02 4.4290237e-02
  3.1018938e-04 8.3457333e-01]]
[[3.5067971e-04 7.4422508e-02 6.6936724e-03 3.9359331e-02 4.4290237e-02
  3.1018938e-04 8.3457333e-01]]
[[8.1935987e-06 5.6582485e-05 5.9759791e-04 8.3607774e-05 1.2484782e-04
  1.6907632e-05 9.9911231e-01]]
[[8.6297689e-05 7.9984264e-03 2.4463807e-03 3.0876892e-03 5.4660235e-03
  6.5891305e-05 9.8084933e-01]]
[[5.3848682e-05 3.4055235e-03 2.0299114e-03 3.2319904e-03 4.5572547e-03
  5.1256207e-05 9.8667020e-01]]
[[1.0072247e-04 7.7744536e-03 3.2669192e-03 4.2008362e-03 8.5457023e-03
  1.1950895e-04 9.7599185e-01]]
[[1.0072247e-04 7.7744536e-03 3.2669192e-03 4.2008362e-03 8.5457

[[6.8876820e-05 2.0130418e-04 9.9782320e-04 5.2940897e-05 9.0948539e-05
  3.4037043e-04 9.9824774e-01]]
[[6.8876820e-05 2.0130418e-04 9.9782320e-04 5.2940897e-05 9.0948539e-05
  3.4037043e-04 9.9824774e-01]]
[[4.0688501e-05 1.3170873e-04 9.2603953e-04 5.6991033e-05 1.1713801e-04
  2.5914726e-04 9.9846834e-01]]
[[1.57395625e-05 1.43078196e-05 2.81301414e-04 8.83258053e-06
  1.13072829e-05 1.10254114e-04 9.99558270e-01]]
[[8.0218320e-05 6.9907168e-05 1.2588446e-03 8.2334489e-05 5.6046716e-05
  6.4304075e-04 9.9780971e-01]]
[[2.2426131e-05 2.1825950e-05 5.3719850e-04 2.1378992e-05 2.1469134e-05
  1.4080279e-04 9.9923491e-01]]
[[2.2426131e-05 2.1825950e-05 5.3719850e-04 2.1378992e-05 2.1469134e-05
  1.4080279e-04 9.9923491e-01]]
[[2.1947229e-05 4.2981286e-05 5.1229954e-04 2.1098778e-05 3.4749297e-05
  2.3359664e-04 9.9913341e-01]]
[[1.1707709e-05 6.2055892e-06 3.7068274e-04 5.8821738e-06 1.2755065e-05
  6.2217820e-05 9.9953055e-01]]
[[1.2755218e-05 1.3814506e-05 4.0236241e-04 1.1670450e-05

[[0.00098933 0.00116085 0.01960025 0.01797467 0.00192574 0.01126506
  0.9470841 ]]
[[0.00098933 0.00116085 0.01960025 0.01797467 0.00192574 0.01126506
  0.9470841 ]]
[[0.00098647 0.00065582 0.08750935 0.39073128 0.01038284 0.00078451
  0.5089497 ]]
[[0.00098647 0.00065582 0.08750935 0.39073128 0.01038284 0.00078451
  0.5089497 ]]
[[9.6049177e-04 6.9781538e-04 1.1357683e-01 4.7286850e-01 1.6924392e-02
  4.2317918e-04 3.9454877e-01]]
[[6.1479892e-04 3.1467946e-04 5.1167831e-02 8.1828266e-02 4.0442557e-03
  4.9637788e-04 8.6153388e-01]]
[[6.1479892e-04 3.1467946e-04 5.1167831e-02 8.1828266e-02 4.0442557e-03
  4.9637788e-04 8.6153388e-01]]
[[8.7628514e-04 1.7382998e-04 7.7979319e-02 4.0157296e-02 2.6928526e-03
  7.7457324e-04 8.7734586e-01]]
[[8.7628514e-04 1.7382998e-04 7.7979319e-02 4.0157296e-02 2.6928526e-03
  7.7457324e-04 8.7734586e-01]]
[[1.3905598e-03 3.5394760e-04 1.1382922e-01 2.4773845e-01 7.6704202e-03
  1.1796132e-03 6.2783784e-01]]
[[7.11102854e-04 1.92547304e-04 4.55228873e-

[[4.8151752e-04 3.2794233e-05 1.3664149e-01 8.3445217e-03 1.8406087e-03
  1.1111728e-03 8.5154796e-01]]
[[1.5910295e-03 3.9427996e-05 5.9870201e-01 1.0836275e-02 4.0420420e-03
  3.5417308e-03 3.8124749e-01]]
[[1.3055730e-04 1.0246504e-04 2.5539352e-02 1.3997400e-02 2.1573554e-03
  1.2916062e-04 9.5794362e-01]]
[[1.3055730e-04 1.0246504e-04 2.5539352e-02 1.3997400e-02 2.1573554e-03
  1.2916062e-04 9.5794362e-01]]
[[1.1643792e-03 3.0387801e-03 4.8309874e-02 1.1673356e-02 1.3931928e-02
  6.9361099e-04 9.2118812e-01]]
[[1.1643792e-03 3.0387801e-03 4.8309874e-02 1.1673356e-02 1.3931928e-02
  6.9361099e-04 9.2118812e-01]]
[[5.4155022e-05 9.1319394e-05 1.9918855e-03 2.4292388e-04 2.4730535e-04
  2.4800835e-05 9.9734759e-01]]
[[5.4155022e-05 9.1319394e-05 1.9918855e-03 2.4292388e-04 2.4730535e-04
  2.4800835e-05 9.9734759e-01]]
[[8.3979845e-05 1.4101609e-04 5.1421011e-03 3.2084019e-04 1.0483930e-03
  8.2333536e-05 9.9318141e-01]]
[[4.14923397e-06 3.02939179e-06 1.71971624e-03 1.15622825e-04
  